In [1]:
!wandb login

wandb: Currently logged in as: 202316187 (sergom2). Use `wandb login --relogin` to force relogin


In [3]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import wandb

# Configuración inicial de wandb
wandb.init(project="ML2-img-class", name="EfficientNet_B4_Model", config={
    "learning_rate": 0.00005,
    "architecture": "efficientnet_b4",
    "dataset": "Tu Dataset Local",
    "unfreezed_layers": 15,
    "epochs": 20,
    "batch_size": 128
})
config = wandb.config

# Transformaciones
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(380),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.Resize(450),
    transforms.CenterCrop(380),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Carga de datos
train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=config.batch_size, shuffle=False)

# Cargar el modelo preentrenado
model = models.efficientnet_b4(pretrained=True)

# Descongelar las últimas 15 capas
for child in list(model.children())[-config.unfreezed_layers:]:
    for param in child.parameters():
        param.requires_grad = True

# Cambiar la capa final para 15 clases
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 15)

# Criterio de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        start_time_epoch = time.time()
        
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_accuracy += (outputs.argmax(1) == labels).float().mean().item() * inputs.size(0)
            
            if i % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {loss.item()}')
        
        epoch_duration = time.time() - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f'Estimated time left: {estimated_time_left/60:.2f} minutes')
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = running_accuracy / len(train_loader.dataset)
        
        # Validación
        model.eval()
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * inputs.size(0)
                val_running_accuracy += (outputs.argmax(1) == labels).float().mean().item() * inputs.size(0)
                
        val_epoch_loss = val_running_loss / len(valid_loader.dataset)
        val_epoch_accuracy = val_running_accuracy / len(valid_loader.dataset)
        
        scheduler.step()
        
        # Log to W&B
        wandb.log({
            'train_loss': epoch_loss, 
            'train_accuracy': epoch_accuracy,
            'val_loss': val_epoch_loss, 
            'val_accuracy': val_epoch_accuracy
        })
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_epoch_accuracy:.4f}')
               
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=config.epochs)
torch.save(model.state_dict(), 'efficientnet_b4_model.pth')


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to C:\Users\sergi/.cache\torch\hub\checkpoints\efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:02<00:00, 26.7MB/s]


Epoch 1, Batch 1, Loss: 2.709487199783325
Epoch 1, Batch 11, Loss: 2.6965267658233643
Epoch 1, Batch 21, Loss: 2.6925296783447266
Epoch 1, Batch 31, Loss: 2.657621383666992
Epoch 1, Batch 41, Loss: 2.644847869873047
Estimated time left: 3478.87 minutes
Epoch 1/20, Train Loss: 2.6762, Train Accuracy: 0.1759, Val Loss: 2.6410, Val Accuracy: 0.3433
Epoch 2, Batch 1, Loss: 2.635890007019043
Epoch 2, Batch 11, Loss: 2.6136369705200195
Epoch 2, Batch 21, Loss: 2.5777878761291504
Epoch 2, Batch 31, Loss: 2.562704563140869
Epoch 2, Batch 41, Loss: 2.5257136821746826
Estimated time left: 3057.29 minutes
Epoch 2/20, Train Loss: 2.5734, Train Accuracy: 0.4328, Val Loss: 2.4938, Val Accuracy: 0.5927
Epoch 3, Batch 1, Loss: 2.5089340209960938
Epoch 3, Batch 11, Loss: 2.4094855785369873
Epoch 3, Batch 21, Loss: 2.3960065841674805


In [2]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import wandb

# Configuración inicial de wandb
wandb.init(project="ML2-img-class", name="DenseNet201_Model", config={
    "learning_rate": 0.0001,
    "architecture": "densenet201",
    "dataset": "Tu Dataset Local",
    "unfreezed_layers": 10,
    "epochs": 10,
    "batch_size": 64
})
config = wandb.config

# Transformaciones
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Carga de datos
train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=config.batch_size, shuffle=False)

# Cargar el modelo preentrenado
model = models.densenet201(pretrained=True)

# Descongelar las últimas 15 capas
for param in model.features.parameters():
    param.requires_grad = False

num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 15)  # Cambiado para adaptar a 15 clases

# Criterio de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        start_time_epoch = time.time()
        
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_accuracy += (outputs.argmax(1) == labels).float().mean().item() * inputs.size(0)
            
            if i % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {loss.item()}')
        
        epoch_duration = time.time() - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f'Estimated time left: {estimated_time_left/60:.2f} minutes')
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = running_accuracy / len(train_loader.dataset)
        
        # Validación
        model.eval()
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * inputs.size(0)
                val_running_accuracy += (outputs.argmax(1) == labels).float().mean().item() * inputs.size(0)
                
        val_epoch_loss = val_running_loss / len(valid_loader.dataset)
        val_epoch_accuracy = val_running_accuracy / len(valid_loader.dataset)
        
        scheduler.step()
        
        # Log to W&B
        wandb.log({
            'train_loss': epoch_loss, 
            'train_accuracy': epoch_accuracy,
            'val_loss': val_epoch_loss, 
            'val_accuracy': val_epoch_accuracy
        })
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_epoch_accuracy:.4f}')
               
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=config.epochs)
torch.save(model.state_dict(), 'densenet201_model.pth')


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Batch 1, Loss: 2.7377614974975586
Epoch 1, Batch 11, Loss: 2.627082109451294
Epoch 1, Batch 21, Loss: 2.626823902130127
Epoch 1, Batch 31, Loss: 2.6241648197174072
Epoch 1, Batch 41, Loss: 2.454120635986328
Estimated time left: 55.87 minutes
Epoch 1/10, Train Loss: 2.5566, Train Accuracy: 0.1843, Val Loss: 2.4553, Val Accuracy: 0.2353
Epoch 2, Batch 1, Loss: 2.4415924549102783
Epoch 2, Batch 11, Loss: 2.320944309234619
Epoch 2, Batch 21, Loss: 2.231729745864868
Epoch 2, Batch 31, Loss: 2.227280378341675
Epoch 2, Batch 41, Loss: 2.1489882469177246
Estimated time left: 49.84 minutes
Epoch 2/10, Train Loss: 2.2496, Train Accuracy: 0.3963, Val Loss: 2.1614, Val Accuracy: 0.4153
Epoch 3, Batch 1, Loss: 2.1050400733947754
Epoch 3, Batch 11, Loss: 2.0685713291168213
Epoch 3, Batch 21, Loss: 2.0572926998138428
Epoch 3, Batch 31, Loss: 1.914212942123413
Epoch 3, Batch 41, Loss: 2.0579891204833984
Estimated time left: 42.00 minutes
Epoch 3/10, Train Loss: 2.0094, Train Accuracy: 0.5216,

In [4]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import wandb
import time

# Configuración inicial de wandb
wandb.init(project="ML2-img-class", name="ResNet121_Model", config={
    "learning_rate": 0.0001,
    "architecture": "resnet152",
    "dataset": "Tu Dataset Local",
    "unfreezed_layers": 10,  # Número de capas a descongelar
    "epochs": 10,
})
config = wandb.config

# Transformaciones
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Carga de datos
train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=128, shuffle=False)

# Cargar el modelo preentrenado ResNet152
model = models.resnet152(pretrained=True)

# Descongelar las últimas 10 capas
children = list(model.children())
num_children = len(children)
for i in range(num_children - 10, num_children):
    for param in children[i].parameters():
        param.requires_grad = True

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct / total

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        
        start_time_epoch = time.time()  # Tiempo de inicio de la época
        
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)
            
            if i % 10 == 0:  # Imprime cada 10 lotes
                print(f'Época {epoch+1}, Lote {i+1}, Pérdida del Lote: {loss.item()}')
        
        end_time_epoch = time.time()  # Tiempo al final de la época
        epoch_duration = end_time_epoch - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f"Tiempo estimado restante: {estimated_time_left/60:.2f} minutos")
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = running_accuracy / len(train_loader.dataset)
        
        # Validación
        model.eval()
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * inputs.size(0)
                val_running_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)
                
        val_epoch_loss = val_running_loss / len(valid_loader.dataset)
        val_epoch_accuracy = val_running_accuracy / len(valid_loader.dataset)
        
        scheduler.step()
        
        # Log a W&B
        wandb.log({'train_loss': epoch_loss, 'train_accuracy': epoch_accuracy,
                   'val_loss': val_epoch_loss, 'val_accuracy': val_epoch_accuracy})
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_epoch_accuracy:.4f}')
               
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=config.epochs)
torch.save(model.state_dict(), 'resnet152_model.pth')


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\sergi/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:05<00:00, 40.9MB/s] 


Época 1, Lote 1, Pérdida del Lote: 2.8154356479644775
Época 1, Lote 11, Pérdida del Lote: 1.1189895868301392
Época 1, Lote 21, Pérdida del Lote: 0.6396958231925964
Tiempo estimado restante: 543.07 minutos
Epoch 1/10, Train Loss: 1.2160, Train Accuracy: 0.6888, Val Loss: 0.3733, Val Accuracy: 0.8920
Época 2, Lote 1, Pérdida del Lote: 0.4634690582752228
Época 2, Lote 11, Pérdida del Lote: 0.33883050084114075
Época 2, Lote 21, Pérdida del Lote: 0.35004615783691406
Tiempo estimado restante: 503.67 minutos
Epoch 2/10, Train Loss: 0.3943, Train Accuracy: 0.8697, Val Loss: 0.2633, Val Accuracy: 0.9127
Época 3, Lote 1, Pérdida del Lote: 0.3483724594116211
Época 3, Lote 11, Pérdida del Lote: 0.236895352602005
Época 3, Lote 21, Pérdida del Lote: 0.2205365151166916
Tiempo estimado restante: 455.03 minutos
Epoch 3/10, Train Loss: 0.2841, Train Accuracy: 0.9072, Val Loss: 0.2291, Val Accuracy: 0.9273
Época 4, Lote 1, Pérdida del Lote: 0.19069203734397888
Época 4, Lote 11, Pérdida del Lote: 0.260549